## A neural network to solve the DC-OPF

### Data processing

In [72]:
# Importing the libraries
import torch
from torchviz import make_dot
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [73]:
# from collections import defaultdict
# split a univariate sequence into samples
def split_sequenceUStep(sequence, n_steps_in):
    X, y = list(), list()
    # X, y = defaultdict(list), defaultdict(list)
    
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [74]:
# Define the study case
CaseName = '3-bus'

In [75]:
# Load the data
df_dict_gen   = pd.read_csv('01.Data/'+ CaseName+'/ANN_Dict_Generation_'        +CaseName+'.csv', header=0, index_col=[0])
df_data_gen   = pd.read_csv('01.Data/'+ CaseName+'/ANN_Data_Generation_'        +CaseName+'.csv', header=0, index_col=[0])
df_demand     = pd.read_csv('01.Data/'+ CaseName+'/ANN_Data_Demand_'            +CaseName+'.csv', header=0, index_col=[0,1,2])
df_generation = pd.read_csv('01.Data/'+ CaseName+'/ANN_Result_GenerationEnergy_'+CaseName+'.csv', header=0, index_col=[0,1,2])

In [76]:
df_data_gen['MinimumPower']['CCGT_1']

0.0

In [77]:
df_demand     = df_demand.stack().reset_index().pivot_table(index=['level_2'], columns=['level_3'], values=0, aggfunc='sum')
df_generation = df_generation.reset_index().pivot_table(index=['LoadLevel'], columns='Unit', values='GWh', aggfunc='sum')

In [78]:
df_data = pd.concat([df_demand, df_generation], axis=1)

In [79]:
# normalize the dataset
# df_data = (df_data - df_data.min()) / (df_data.max() - df_data.min())
df_data['Node_1'] /= 1000
df_data['Node_2'] /= 1000
df_data['Node_3'] /= 1000

In [80]:
# from GWh to MWh
df_data *= 1000

In [81]:
# normalize the demand dataset
df_data['Node_1'] = (df_data['Node_1'] - df_data['Node_1'].min()) / (df_data['Node_1'].max() - df_data['Node_1'].min())
df_data['Node_2'] = (df_data['Node_2'] - df_data['Node_2'].min()) / (df_data['Node_2'].max() - df_data['Node_2'].min())
df_data['Node_3'] = (df_data['Node_3'] - df_data['Node_3'].min()) / (df_data['Node_3'].max() - df_data['Node_3'].min())

In [82]:
# normalize the generation dataset
df_data['CCGT_1'] = (df_data['CCGT_1'] - df_data_gen['MinimumPower']['CCGT_1']) / (df_data_gen['MaximumPower']['CCGT_1'] - df_data_gen['MinimumPower']['CCGT_1'])
df_data['CCGT_2'] = (df_data['CCGT_2'] - df_data_gen['MinimumPower']['CCGT_2']) / (df_data_gen['MaximumPower']['CCGT_2'] - df_data_gen['MinimumPower']['CCGT_2'])

In [83]:
# split into input (demand) and outputs (generation)
X = df_data.iloc[:,:3].values
y = df_data.iloc[:,3:6].values


In [84]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [85]:
# Convert the data into PyTorch tensors
train_inputs  = torch.from_numpy(X_train)
train_targets = torch.from_numpy(y_train)
test_inputs   = torch.from_numpy(X_test)
test_targets  = torch.from_numpy(y_test)

In [86]:
# Define the neural network model
class aNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super().__init__()
        self.hidden_layer1 = torch.nn.Linear(input_size, hidden_size1)
        torch.nn.init.kaiming_uniform_(self.hidden_layer1.weight, a=0)
        self.hidden_layer2 = torch.nn.Linear(hidden_size1, hidden_size2)
        self.hidden_layer3 = torch.nn.Linear(hidden_size2, hidden_size3)
        self.output_layer = torch.nn.Linear(hidden_size3, output_size)

        # define the device to use (GPU or CPU)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def forward(self, input):
        hidden1 = torch.relu(self.hidden_layer1(input))
        hidden2 = torch.relu(self.hidden_layer2(hidden1))
        # hidden3 = torch.relu(self.hidden_layer3(hidden2))
        hidden3 = self.hidden_layer3(hidden2)
        output = self.output_layer(hidden3)
        return output

In [87]:
# Initialize the model and optimizer
model = aNN(input_size=3, hidden_size1=32, hidden_size2=16, hidden_size3=8, output_size=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [88]:
# Train the model
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    # Forward pass
    train_predictions = model(train_inputs.float())
    train_loss = torch.nn.MSELoss()(train_predictions.float().squeeze(), train_targets.float())

    # Backward pass
    # optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # Print the training loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}, Train Loss: {train_loss.item()}')

Epoch 10, Train Loss: 0.2950093448162079
Epoch 20, Train Loss: 0.2402355670928955
Epoch 30, Train Loss: 0.18687888979911804
Epoch 40, Train Loss: 0.13246048986911774
Epoch 50, Train Loss: 0.07768283784389496
Epoch 60, Train Loss: 0.030700836330652237
Epoch 70, Train Loss: 0.0064253793098032475
Epoch 80, Train Loss: 0.004464925732463598
Epoch 90, Train Loss: 0.004392119124531746
Epoch 100, Train Loss: 0.002982495818287134
Epoch 110, Train Loss: 0.0027553467079997063
Epoch 120, Train Loss: 0.0024625477381050587
Epoch 130, Train Loss: 0.0021653929725289345
Epoch 140, Train Loss: 0.001939917798154056
Epoch 150, Train Loss: 0.0017709741368889809
Epoch 160, Train Loss: 0.0016351070953533053
Epoch 170, Train Loss: 0.0015163097996264696
Epoch 180, Train Loss: 0.0014137565158307552
Epoch 190, Train Loss: 0.0013234467478469014
Epoch 200, Train Loss: 0.0012428901391103864
Epoch 210, Train Loss: 0.0011697628069669008
Epoch 220, Train Loss: 0.0011025756830349565
Epoch 230, Train Loss: 0.00104022817

In [89]:
# Evaluate the model on the test set
test_predictions = model(test_inputs.float())
test_loss = torch.nn.MSELoss()(test_predictions.float().squeeze(), test_targets.float())
print(f'Test Loss: {test_loss.item()}')

Test Loss: 1.099209111998789e-05


In [90]:
test_predictions.float()

tensor([[ 1.3736e-01,  9.9929e-01],
        [-5.7791e-04,  8.0667e-01],
        [ 2.4603e-02,  9.9406e-01],
        ...,
        [-4.3851e-04,  7.3671e-01],
        [ 1.7682e-01,  9.9639e-01],
        [-3.4897e-03,  9.9056e-01]], grad_fn=<AddmmBackward0>)

In [91]:
# Save the trained model
torch.save(model.state_dict(), "model.pth")

In [92]:
# Convert the test predictions and test output to NumPy arrays
test_targets = test_targets.detach().numpy()
test_predictions_s = test_predictions.detach().numpy()

In [93]:
test_targets_df = pd.DataFrame(test_targets, columns=['G1_target','G2_target'])
test_targets_df['G1_target'] = test_targets_df['G1_target'] * (df_data_gen['MaximumPower']['CCGT_1'] - df_data_gen['MinimumPower']['CCGT_1']) + df_data_gen['MinimumPower']['CCGT_1']
test_targets_df['G2_target'] = test_targets_df['G2_target'] * (df_data_gen['MaximumPower']['CCGT_2'] - df_data_gen['MinimumPower']['CCGT_2']) + df_data_gen['MinimumPower']['CCGT_2']
test_targets_df.to_csv('test_targets.csv', index=False)
test_predictions_df = pd.DataFrame(test_predictions_s, columns=['G1_estimate','G2_estimate'])
test_predictions_df['G1_estimate'] = test_predictions_df['G1_estimate'] * (df_data_gen['MaximumPower']['CCGT_1'] - df_data_gen['MinimumPower']['CCGT_1']) + df_data_gen['MinimumPower']['CCGT_1']
test_predictions_df['G2_estimate'] = test_predictions_df['G2_estimate'] * (df_data_gen['MaximumPower']['CCGT_2'] - df_data_gen['MinimumPower']['CCGT_2']) + df_data_gen['MinimumPower']['CCGT_2']
test_predictions_df.to_csv('test_predictions.csv', index=False)

In [94]:
time_df = pd.DataFrame({'LoadLevel': pd.date_range(start='2023-05-04 00:00:00', periods=len(test_predictions_df), freq='H')})
frames = [time_df, test_predictions_df, test_targets_df]
result = pd.concat(frames, axis=1).set_index('LoadLevel')

In [95]:
source = result.stack().reset_index().rename(columns={'level_1':'Demand', 0:'Value'})

In [96]:
import altair as alt

lines = (
    alt.Chart(source)
    .mark_line()
    .encode(x="LoadLevel", y="Value", color="Demand")
).properties(width=1500, height=500)
lines.save('Plot.html', embed_options={'renderer':'svg'})

In [97]:
## An extensive graph of the model using torchviz
# from torchviz import make_dot

# dot = make_dot(test_predictions, params=dict(model.named_parameters()))
# dot.render("rnn_torchviz", format="png")
# dot.view()

In [98]:
# plot the model graph using torchview
from torchview import draw_graph
model_graph = draw_graph(model, input_size=(test_inputs.size()), device='meta', save_graph=True, graph_name='model_graph')


(dot.exe:21088): Pango-WARNING **: couldn't load font "Linux libertine Not-Rotated 10", falling back to "Sans Not-Rotated 10", expect ugly output.
